In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm

In [3]:
#central
#把所有区域的好车数量往N/A调整
#在这种情况下可能存在的一种情况是，因为
# initialize parameters
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm
import time
from multiprocessing import Pool

R = {
        'lambda0' : 20.0,
        'lambda1' : 20.0,
        'lambda2' : 20.0,
        'lambda3' : 20.0,
        'car_move': 10.0,
        'capacity': 12,
        'gamma' : 5.0,
        'phi' : 5.0,
        'delta' : 5.0,
        'broken' : 0.15,
        'period' : 90, #the maximal time for a carrier to move among areas
       }
#queues: A0, A1, C, F, D, R00, R01, R10, R11
P = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]

# real
# the influence of phi
# in this version, carrier moves among areas to collect broken bikes
# then brings them to the repair center, and bring the normal bikes back to areas
class BikeNet():
    def __init__(self, N, A, R, P, warmup_time, run_time):
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time
        #self.C = 0
        self.F = 0
        #self.D = 0
        self.capacity = self.R['capacity']
        self.areas = list(range(A))

        self.served_customers = 0
        self.target_number = int(self.N/self.A)
        
    def reset(self):
        self.T = 0
#         self.C = 0
        self.F = 0
#         self.D = 0
        self.served_customers = 0
        #queues: A0, A1, B0, B1, FN, FB, R00, R01, R10, R11
        self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+2+self.A)
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        #event of the carrier, [time, kind, place, [normal, broken, time]]
        heapq.heappush(self.scheduler, [0, 0, 0, [0,0,self.T+self.R['period']]])
        #return self.state + [self.T]

    def simulate(self, writefile, start_index):
        if writefile:
            with open(RESULT_ADD+str(start_index) + 'move_broken_server_4ps.csv', 'a', newline='') as fout:
                writer = csv.writer(fout)
                #writer.writerow(self.R)
                #writer.writerow(self.R.values())
                #writer.writerow(self.reset())
                s = 0
                for i in range(15):
                    self.reset()
                    while self.T <= self.time_limit:
                        self.step()
                    s += self.served_customers / (self.T-self.warmup_time)
                writer.writerow([start_index, self.R['broken'], self.R['car_move'], s/15])
        else:
            self.reset()
            while self.T <= self.time_limit:
                self.step()
            return self.served_customers / (self.T-self.warmup_time)
                
    def get_riding_time(self, s, t):
        '''
        |0|1|
        |3|2|
        '''
        if s==t: r = 0.5
        elif abs(s-t) == 2: r = 2.0
        else: r = 1.0
        return random.expovariate(r)
                
    def get_index(self, target):
        if target == 'c': return 4
        elif target == 'f': return 5
        elif target == 'd': return 6
        else:
            s, t = int(target[0]), int(target[1])
            return 2*self.A + 2 + 4*s + t
        
    def add_event(self, kind, s):
        if kind == 2:
            next_time = random.expovariate(self.R['gamma']) + self.T
            start, end = s, 'f'
        elif kind == 3:
            next_time = random.expovariate(self.R['phi']) + max(self.T, self.F)
            self.F, start, end = next_time, 'f', 'd'
        elif kind == 0:
            next_time = random.expovariate(self.R['delta']) + self.T
            start, end = random.choice(list(range(self.A))), [s, 0, self.T+self.R['period']]
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''kind:
        -1: customer ride a bike away
         0: carrier arrives as a area
         1: a bike arrives
         2: carrier full of broken bikes arrives as the repairing center
         3: a bike was fixed
        '''
        if kind == 0: #carrier 正在areas之间逡巡中，鱼戏莲叶南，鱼戏莲叶北
            '''
            处理的顺序为：
            1、如果当前区域好车数不到target_number，则将好车放下，之后pick up坏车
            2、如果超过目标值，优先将坏车pick up，剩余的位置pick up好车
            '''
            normal, broken, end_time = terminal[0], terminal[1], terminal[2]
            d = self.target_number-self.state[start]
            if d>0:
                #drop down broken bikes
                number = min(normal, d)
                self.state[start] += number
                self.scheduler[0][3][0] -= number
                normal -= number
                #pick up normal bikes
                number = min(self.capacity - broken-normal, self.state[self.A + start])
                self.state[self.A+start] -= number
                self.scheduler[0][3][1] += number
                broken += number
            elif d<0:
                #pick up broken bikes
                number = min(self.capacity - broken-normal, self.state[self.A + start])
                self.state[self.A+start] -= number
                self.scheduler[0][3][1] += number
                broken += number
                #drop down normal bikes
                if self.capacity > normal+broken:
                    number = min(self.state[start] - self.target_number, self.capacity-normal-broken)
                    self.state[start] -= number
                    self.scheduler[0][3][0] += number
                    normal += number
            if broken == self.capacity:
                #print('carrier goes back')
                heapq.heappop(self.scheduler)
                self.add_event(2, broken)
            else:
                self.scheduler[0][0] += random.expovariate(self.R['car_move'])
                self.scheduler[0][2] = (start+1)%4
                heapq.heapify(self.scheduler) 
        elif kind == 1: # 顾客骑行到达
            self.state[self.get_index(start)] -= 1
            if random.random()<self.R['broken']:
                self.state[self.A+terminal] += 1
                heapq.heappop(self.scheduler)
            else:
                self.state[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            if self.state[self.A * 2 + 1]==0:
                self.add_event(3, start)
            self.state[self.A * 2 + 1] += start
            heapq.heappop(self.scheduler)
            number = min(self.capacity, self.state[self.A * 2])
            self.add_event(0, number)
            self.state[self.A * 2] -= number
        elif kind == 3:
            #print(self.state)
            self.state[self.A * 2] += 1
            self.state[self.A * 2 + 1] -= 1
            heapq.heappop(self.scheduler)
            if self.state[self.A * 2 + 1] > 0:
                self.add_event(3, start)
                #print('next fix')
            #print(self.state)
        else:# 顾客到达
            if self.state[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                target = str(start)+ str(np.random.choice(self.areas, 1, p=self.P[start])[0])
                if self.T>self.warmup_time: self.served_customers += 1
                self.state[start] -= 1
                self.state[self.get_index(target)] += 1
                heapq.heappop(self.scheduler)
                next_time = self.get_riding_time(int(target[0]), int(target[1])) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, target, int(target[1])])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


        return self.state+[self.T]

N = 100  # total number of bikes in the QN
A = 4  # A for areas, indicates the number of areas and the action space


def run(start):

    P0 = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]
    P1 = [
        [0.1, 0.2, 0.3, 0.4], 
        [0.4, 0.1, 0.2, 0.3], 
        [0.3, 0.4, 0.1, 0.2],
        [0.2, 0.3, 0.4, 0.1],
      ]
    P2 = [[0.1, 0.2, 0.3, 0.4],
        [0.3, 0.4, 0.1, 0.2],
        [0.2, 0.3, 0.4, 0.1],
        [0.4, 0.1, 0.2, 0.3]]
    Pr = [[0.07, 0.42, 0.38, 0.13],
        [0.21, 0.19, 0.27, 0.33],
        [0.07, 0.02, 0.6, 0.31],
        [0.395, 0.001, 0.231, 0.373]]
    P = [P0, P1, P2, Pr]
        
    env = BikeNet(N=N,
                  A=A,
                  R=R,
                  P=P,
                  warmup_time=WARMUP_TIME,
                  run_time=RUN_TIME)

    #random.seed(1)
    env.P = P[start]
    for b in tqdm(range(1,51)):
        env.R['broken'] = b * 0.02
        env.simulate(IF_WRITEFILE, start)
    
# Hyper parameters
EPISODES = 50 # times every setting is replicated
IF_WRITEFILE = True
#IF_WRITEFILE = False
RESULT_ADD = '/Users/valarian/Desktop/article review/Backup server/real/brokenRate/'

WARMUP_TIME = 500
RUN_TIME = 1000
    
if __name__ == '__main__':
    
    
    p = Pool(2)
    p.map(run, range(2,4))

100%|██████████| 50/50 [16:04<00:00, 19.28s/it]


In [5]:
#central
#把所有区域的好车数量往N/A调整
#在这种情况下可能存在的一种情况是，因为
# initialize parameters
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm
import time
from multiprocessing import Pool

R = {
        'lambda0' : 20.0,
        'lambda1' : 20.0,
        'lambda2' : 20.0,
        'lambda3' : 20.0,
        'car_move': 1.0,
        'capacity': 12,
        'gamma' : 5.0,
        'phi' : 5.0,
        'delta' : 5.0,
        'broken' : 0.15,
        'episode' : 20,
        'period' : 60,
       }
#queues: A0, A1, C, F, D, R00, R01, R10, R11
P = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]

# real
# the influence of phi
# in this version, carrier moves among areas to collect broken bikes
# then brings them to the repair center, and bring the normal bikes back to areas
class BikeNet():
    def __init__(self, N, A, R, P, warmup_time, run_time):
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time
        #self.C = 0
        self.F = 0
        #self.D = 0
        self.capacity = self.R['capacity']
        self.areas = list(range(A))

        self.served_customers = 0
        self.target_number = int(self.N/self.A)
        
    def reset(self):
        self.T = 0
#         self.C = 0
        self.F = 0
#         self.D = 0
        self.served_customers = 0
        #queues: A0, A1, B0, B1, FN, FB, R00, R01, R10, R11
        self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+2+self.A)
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        #event of the carrier, [time, kind, place, [normal, broken, time]]
        heapq.heappush(self.scheduler, [0, 0, 0, [0,0,self.T+self.R['period']]])

    def simulate(self, writefile, start_index):
        if writefile:
            with open(RESULT_ADD+str(start_index) + 'real_Nlambda_4ps.csv', 'a', newline='') as fout:
                writer = csv.writer(fout)
                #writer.writerow(self.R)
                #writer.writerow(self.R.values())
                #writer.writerow(self.reset())
                s = 0
                for i in range(20):
                    self.reset()
                    while self.T <= self.time_limit:
                        self.step()
                    s += self.served_customers / (self.T-self.warmup_time)
                writer.writerow([start_index, self.R['lambda0'], self.N, s/20])
        else:
            s=0
            for i in range(20):
                self.reset()
                while self.T <= self.time_limit:
                    self.step()
                s += self.served_customers / (self.T-self.warmup_time)
            #return self.served_customers / (self.T-self.warmup_time)
            print(s/20)
                
    def get_riding_time(self, s, t):
        '''
        |0|1|
        |3|2|
        '''
        if s==t: r = 0.5
        elif abs(s-t) == 2: r = 2.0
        else: r = 1.0
        return random.expovariate(r)
                
    def get_index(self, target):
        if target == 'c': return 4
        elif target == 'f': return 5
        elif target == 'd': return 6
        else:
            s, t = int(target[0]), int(target[1])
            return 2*self.A + 2 + 4*s + t
        
    def add_event(self, kind, s):
        if kind == 2:
            next_time = random.expovariate(self.R['gamma']) + self.T
            start, end = s, 'f'
        elif kind == 3:
            next_time = random.expovariate(self.R['phi']) + max(self.T, self.F)
            self.F, start, end = next_time, 'f', 'd'
        elif kind == 0:
            next_time = random.expovariate(self.R['delta']) + self.T
            start, end = random.choice(list(range(self.A))), [s, 0, self.T+self.R['period']]
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''kind:
        -1: customer ride a bike away
         0: carrier arrives as a area
         1: a bike arrives
         2: carrier full of broken bikes arrives as the repairing center
         3: a bike was fixed
        '''
        if kind == 0: #carrier 正在areas之间逡巡中，鱼戏莲叶南，鱼戏莲叶北
            '''
            处理的顺序为：
            1、如果当前区域好车数不到target_number，则将好车放下，之后pick up坏车
            2、如果超过目标值，优先将坏车pick up，剩余的位置pick up好车
            '''
            normal, broken, end_time = terminal[0], terminal[1], terminal[2]
            d = self.target_number-self.state[start]
            if d>0:
                #drop down broken bikes
                number = min(normal, d)
                self.state[start] += number
                self.scheduler[0][3][0] -= number
                normal -= number
                #pick up normal bikes
                number = min(self.capacity - broken-normal, self.state[self.A + start])
                self.state[self.A+start] -= number
                self.scheduler[0][3][1] += number
                broken += number
            elif d<0:
                #pick up broken bikes
                number = min(self.capacity - broken-normal, self.state[self.A + start])
                self.state[self.A+start] -= number
                self.scheduler[0][3][1] += number
                broken += number
                #drop down normal bikes
                if self.capacity > normal+broken:
                    number = min(self.state[start] - self.target_number, self.capacity-normal-broken)
                    self.state[start] -= number
                    self.scheduler[0][3][0] += number
                    normal += number
            if broken == self.capacity or (self.T>end_time and normal==0):
                #print('carrier goes back')
                heapq.heappop(self.scheduler)
                self.add_event(2, broken)
            else:
                self.scheduler[0][0] += random.expovariate(self.R['car_move'])
                self.scheduler[0][2] = (start+1)%4
                heapq.heapify(self.scheduler) 
        elif kind == 1: # 顾客骑行到达
            self.state[self.get_index(start)] -= 1
            if random.random()<self.R['broken']:
                self.state[self.A+terminal] += 1
                heapq.heappop(self.scheduler)
            else:
                self.state[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            if self.state[self.A * 2 + 1]==0:
                self.add_event(3, start)
            self.state[self.A * 2 + 1] += start
            heapq.heappop(self.scheduler)
            number = min(self.capacity, self.state[self.A * 2])
            self.add_event(0, number)
            self.state[self.A * 2] -= number
        elif kind == 3:
            #print(self.state)
            self.state[self.A * 2] += 1
            self.state[self.A * 2 + 1] -= 1
            heapq.heappop(self.scheduler)
            if self.state[self.A * 2 + 1] > 0:
                self.add_event(3, start)
                #print('next fix')
            #print(self.state)
        else:# 顾客到达
            if self.state[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                target = str(start)+ str(np.random.choice(self.areas, 1, p=self.P[start])[0])
                if self.T>self.warmup_time: self.served_customers += 1
                self.state[start] -= 1
                self.state[self.get_index(target)] += 1
                heapq.heappop(self.scheduler)
                next_time = self.get_riding_time(int(target[0]), int(target[1])) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, target, int(target[1])])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


N = 100  # total number of bikes in the QN
A = 4  # A for areas, indicates the number of areas and the action space


def run(start):

    P0 = [
        [0.1, 0.4, 0.4, 0.1], 
        [0.4, 0.1, 0.1, 0.4], 
        [0.4, 0.1, 0.1, 0.4],
        [0.1, 0.4, 0.4, 0.1],
      ]
    P1 = [
        [0.1, 0.2, 0.3, 0.4], 
        [0.4, 0.1, 0.2, 0.3], 
        [0.3, 0.4, 0.1, 0.2],
        [0.2, 0.3, 0.4, 0.1],
      ]
    P2 = [[0.1, 0.2, 0.3, 0.4],
        [0.3, 0.4, 0.1, 0.2],
        [0.2, 0.3, 0.4, 0.1],
        [0.4, 0.1, 0.2, 0.3]]
    Pr = [[0.07, 0.42, 0.38, 0.13],
        [0.21, 0.19, 0.27, 0.33],
        [0.07, 0.02, 0.6, 0.31],
        [0.395, 0.001, 0.231, 0.373]]
    P = [P0, P1, P2, Pr]
        
    env = BikeNet(N=N,
                  A=A,
                  R=R,
                  P=P,
                  warmup_time=WARMUP_TIME,
                  run_time=RUN_TIME)

    #random.seed(1)
    env.P = P[start]
    #for l in tqdm(range(1,61)):
    for l in range(1,2):
        env.R['lambda0'] =env.R['lambda1'] = env.R['lambda2'] = env.R['lambda3'] = l
        #for n in [1,2]:
        for n in [23]:#,24,25,26,27,28,47,48,49,50,51,52]:
            env.N = n
            env.simulate(IF_WRITEFILE, start)
    
# Hyper parameters
EPISODES = 50 # times every setting is replicated
#IF_WRITEFILE = True
IF_WRITEFILE = False
RESULT_ADD = 'C:/Rebalancing/data/simulationResult/central/data/real_Nlambda_4ps/'

WARMUP_TIME = 500
RUN_TIME = 1000
    
if __name__ == '__main__':
    
    run(0)
#     p = Pool(4)
#     p.map(run, range(4))

1.1224778376103584


In [18]:
random.expovariate(200)

0.009067348396679931

In [14]:
[[0.21270073149161792, 0.16878458237092583, 0.35541984038084723, 0.26309484575660896], [0.3456978275629366, 0.0471578439844552, 0.29850863145956646, 0.30863569699304166], [0.2110639509078739, 0.4328140690070925, 0.34410603368836773, 0.012015946396665905], [0.06096431887945661, 0.23983612249852623, 0.5191434783416946, 0.18005608028032263]]

[[0.21270073149161792,
  0.16878458237092583,
  0.35541984038084723,
  0.26309484575660896],
 [0.3456978275629366,
  0.0471578439844552,
  0.29850863145956646,
  0.30863569699304166],
 [0.2110639509078739,
  0.4328140690070925,
  0.34410603368836773,
  0.012015946396665905],
 [0.06096431887945661,
  0.23983612249852623,
  0.5191434783416946,
  0.18005608028032263]]